In [ ]:
import pandas as pd

# Load all the processed datasets
file_paths = {
"medications": "ProcessedData/processed_medications.csv",
"observations": "ProcessedData/processed_observations.csv",
"patients": "ProcessedData/processed_patients.csv",
"allergies": "ProcessedData/processed_allergies.csv",
"careplans": "ProcessedData/processed_careplans.csv",
"conditions": "ProcessedData/processed_conditions.csv",
"immunizations": "ProcessedData/processed_immunizations.csv",
}

# Read the files and rename duplicate columns
data_frames = {}
for key, path in file_paths.items():
    df = pd.read_csv(path)
    # Rename columns by adding the filename as a suffix to avoid duplicates
    df.columns = [f"{col}_{key}" if col != 'id' else 'id' for col in df.columns]
    data_frames[key] = df

# Merge all dataframes on the 'id' column
merged_data = data_frames["patients"]
for key, df in data_frames.items():
    if key != "patients":
        merged_data = pd.merge(merged_data, df, on="id", how="left")

# Display the structure of the merged dataset
merged_data.head(), merged_data.columns.tolist()


In [ ]:

# Preprocessing the merged data
def preprocess_data(data):
    import numpy as np
    from sklearn.preprocessing import StandardScaler, LabelEncoder

    # Fill missing values
    data.fillna(0, inplace=True)

    # Normalize numerical columns
    numerical_cols = data.select_dtypes(include=[np.number]).columns
    scaler = StandardScaler()
    data[numerical_cols] = scaler.fit_transform(data[numerical_cols])

    # Encode categorical columns
    categorical_cols = data.select_dtypes(include=['object']).columns
    for col in categorical_cols:
        encoder = LabelEncoder()
        data[col] = encoder.fit_transform(data[col])

    return data


In [ ]:

# Updated CNN model to predict COVID-19 mortality risk
def build_cnn_model(input_shape, num_classes):
    from tensorflow.keras.models import Sequential
    from tensorflow.keras.layers import Conv1D, Dense, Dropout, BatchNormalization, Flatten

    model = Sequential([
        Conv1D(64, kernel_size=3, activation='relu', input_shape=input_shape),
        BatchNormalization(),
        Dropout(0.3),
        Conv1D(128, kernel_size=3, activation='relu'),
        BatchNormalization(),
        Dropout(0.3),
        Flatten(),
        Dense(128, activation='relu'),
        Dropout(0.5),
        Dense(num_classes, activation='softmax')
    ])

    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model


In [ ]:

# Prepare data and train the CNN model
def train_model(merged_data):
    from sklearn.model_selection import train_test_split
    from tensorflow.keras.utils import to_categorical
    import numpy as np

    # Define target labels
    def create_labels(data):
        labels = []
        for _, row in data.iterrows():
            if row["DECEASED_patients"] == 1:
                if "COVID-19_careplans" in row and row["COVID-19_careplans"] == 1:
                    labels.append(0)  # Had COVID, died
                else:
                    labels.append(1)  # Didn't have COVID, died
            else:
                if "COVID-19_careplans" in row and row["COVID-19_careplans"] == 1:
                    labels.append(3)  # Had COVID, lived
                else:
                    labels.append(2)  # Didn't have COVID, lived
        return np.array(labels)

    # Preprocess the data
    processed_data = preprocess_data(merged_data)
    labels = create_labels(processed_data)
    processed_data.drop(columns=["DECEASED_patients", "COVID-19_careplans"], inplace=True, errors='ignore')

    # Train-test split
    X_train, X_test, y_train, y_test = train_test_split(processed_data, labels, test_size=0.2, random_state=42)
    y_train, y_test = to_categorical(y_train), to_categorical(y_test)

    # Build the CNN model
    input_shape = (X_train.shape[1], 1)
    model = build_cnn_model(input_shape, y_train.shape[1])

    # Train the model
    model.fit(X_train.values[..., np.newaxis], y_train, epochs=10, batch_size=32, validation_split=0.2)
    return model, X_test, y_test
